In [4]:
import os
os.environ['TRANSFORMERS_CACHE'] = os.path.join(os.getcwd(), 'transformers')
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [5]:
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import logging
import sys

In [6]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

checkpoint = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

model_copy = AutoModelForCausalLM.from_pretrained(checkpoint).to('cuda:0')
# , torch_dtype="auto", device_map="auto"
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.eval()
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [7]:
sentiment_path_1 = "cardiffnlp/twitter-roberta-base-sentiment" # 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
sentiment_path_2 = 'nlptown/bert-base-multilingual-uncased-sentiment'
sentiment_1 = pipeline('sentiment-analysis',model=sentiment_path_1,tokenizer=sentiment_path_1, top_k=None)
sentiment_2 = pipeline('sentiment-analysis',model=sentiment_path_2,tokenizer=sentiment_path_2, top_k=None)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [8]:
sentiment_1("\xa0an (anonymous) (anonymous) (anonymous) (anonymous (anonymous) (anonymous)")

[[{'label': 'LABEL_1', 'score': 0.8726766705513},
  {'label': 'LABEL_0', 'score': 0.0813116580247879},
  {'label': 'LABEL_2', 'score': 0.046011634171009064}]]

In [9]:
sentiment_2("\xa0an (anonymous) (anonymous) (anonymous) (anonymous (anonymous) (anonymous)")

[[{'label': '1 star', 'score': 0.5140755772590637},
  {'label': '2 stars', 'score': 0.22503311932086945},
  {'label': '3 stars', 'score': 0.15606173872947693},
  {'label': '4 stars', 'score': 0.0636562630534172},
  {'label': '5 stars', 'score': 0.041173290461301804}]]

In [10]:
def return_score_nlptown(sentence, percent=1):
    scores = sentiment_2(sentence)[0]
    req_score = 0
    for dict_inner in scores:
        if dict_inner['label'] == '1 star':
            req_score += percent * dict_inner['score']
        if dict_inner['label'] == '2 star':
            req_score += (1 - percent) * dict_inner['score']

    return req_score

def return_score_cardiff_roberta_xlm(sentence):
    scores = sentiment_1(sentence)[0]
    req_score = 0
    for dict_inner in scores:
        if dict_inner['label'] == 'negative':
            req_score += 10 * dict_inner['score']
        if dict_inner['label'] == 'positive':
            req_score -= 10 * dict_inner['score']
        if dict_inner['label'] == 'neutral':
            req_score -= 5 * dict_inner['score']

    return req_score

In [11]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish):  # predicted will be the list of predicted token
        rewards = []
        inputs = tokenizer([input_item['input']], return_tensors="pt").to('cuda:0')
        reply_ids = model_copy.generate(**inputs)
        reply = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
        reply = reply.replace('</s>', '')
        reply = reply.replace('<s>', '')
        reply = reply.strip()
        for i in predicted_list:
            if finish or len(i) > self.env_max_length:
                predicted_text = tokenizer.convert_tokens_to_string(i)
                # reward = 10 * bleu_score.sentence_bleu(references=[word_tokenize(input_item['output'])], hypothesis=word_tokenize(predicted_text))
                # rewards.append(reward)
                reward = return_score_cardiff_roberta_xlm(input_item['input']+predicted_text)
                reward = min(reward, return_score_nlptown(input_item['input']+predicted_text))
                reward += (fuzz.ratio(reply, predicted_text) - 50)
                rewards.append(reward)
                # calculate reward score base on predicted_list
            else:
                rewards.append(0)
        return rewards

In [12]:
import pandas as pd

df = pd.read_csv('../newsCompiled.csv')
df['ol'] = df.apply(lambda x: {'input': x['Input'], 'output': x['Output']}, axis=1)
observation_list = df['ol'].tolist()

In [13]:
observaton_list = [{"input": "Dogecoin is "}]
env = MyRLEnv(
    model, tokenizer, 
    observation_input=observation_list, 
    max_length=25, 
    compare_sample=5
)
actor = TextRLActor(
    env, model, tokenizer, 
    act_deterministically=False, 
    optimizer='adamw',
    temperature=0.8,
    top_k=100,
    top_p=0.85
)
# act_deterministically=False,
# temperature=1.0,
# top_k=0,
# top_p=1.0,
# repetition_penalty=2

In [14]:
agent = actor.agent_ppo(
    update_interval=2, 
    minibatch_size=2, 
    epochs=10,
    lr=3e-4
)

test_phrase = observation_list[0]['input']
print(test_phrase)
actor.predict({'input': test_phrase})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fed official says weak data


/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/transformers/generation/utils.py:1350: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5025

[' will not stop it\n\n\nIn the wake of the US election, the Dow Jones industrial average has dropped by 2.',
 " is cause of fears of new Brexit vote\n\n\nThe UK's economy is still struggling to cope with a huge increase in",
 ' on migration could be a factor\n\n\nJakub Jaitley, the CBI chief, said that the data released',
 ' is a cause for concern\n\n\nThe U.S. government is concerned that the latest findings on the number of people',
 ' means the US is "not ready to fight" the Islamic State\n\n\nThis is the latest in a series of military']

In [15]:
train_agent_with_evaluation(
    agent,
    env,
    steps=100,
    eval_n_steps=None,
    eval_n_episodes=5,
    eval_interval=25,
    outdir='bloom—test-3',
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
S

outdir:bloom—test-3 step:27 episode:0 R:-199
statistics:[('average_value', 1.0506402), ('average_entropy', 1.1968204), ('average_value_loss', 16.410939984954894), ('average_policy_loss', -0.08351403577998667), ('n_updates', 130), ('explained_variance', 0.9966438693589943)]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

evaluation episode 0 length:27 R:-136


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

evaluation episode 3 length:27 R:-194


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

evaluation episode 4 length:27 R:-339
Saved the agent to bloom—test-3/100_finish


(<textrl.actor.TextPPO at 0x7f1c6e45e198>,
 [{'average_value': 1.0506402,
   'average_entropy': 1.1968204,
   'average_value_loss': 16.410939984954894,
   'average_policy_loss': -0.08351403577998667,
   'n_updates': 130,
   'explained_variance': 0.9966438693589943,
   'eval_score': -140.0},
  {'average_value': -1.9027967,
   'average_entropy': 0.61519945,
   'average_value_loss': 83.24562821798027,
   'average_policy_loss': -0.0017791975326834119,
   'n_updates': 270,
   'explained_variance': 1.0,
   'eval_score': -151.6},
  {'average_value': -8.281743,
   'average_entropy': 0.55765826,
   'average_value_loss': 21.89592770934105,
   'average_policy_loss': -0.04957466826215384,
   'n_updates': 400,
   'explained_variance': 0.9141472763773727,
   'eval_score': -137.0},
  {'average_value': -8.6817255,
   'average_entropy': 0.5197389,
   'average_value_loss': 27.645572066307068,
   'average_policy_loss': -0.03527019076142481,
   'n_updates': 500,
   'explained_variance': -6.703717201532655

In [16]:
agent.load('./bloom—test-3/best')
print(observation_list[0])
actor.predict(observation_list[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'input': 'Fed official says weak data', 'output': 'caused by weather, should not slow taper'}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

[' in the in-game in-game in-game in-game in-game in-game in in-game in',
 ' in in-game in-game in-game in-game in-game in-game in in-game in in',
 ' in the which-have-a-chance-to-win-the-game in which-the-neighbor',
 ' in in-game in-game in-game in-game in-game in-game in-game in in-',
 ' in the report in which we in the in-game in-game in-game in-game in-game in-']